In [ ]:








# the network shall also have allocated to each edge the probability of infection        
network=set_weight_degree(network,"positive")
network=set_weight_degree(network,"negative")

k_p=100
k_n=50
S_N=random.choices([v for v in list(network.nodes())],k= k_n)
S_P=CMIA_O_fair_new(network,S_N,k_p,0.01,partition)
print(S_P)


In [1]:
#import of packages
import networkx as nx
import matplotlib.pyplot as plt
import random  
import numpy as np
import sys
from  fractions import Fraction
import timeit
from time import sleep
from collections import defaultdict
import heapq as heap
from tqdm import tqdm
from community import community_louvain
import collections 
import matplotlib.cm as cm
import statistics

from IPython.display import HTML, display
from itertools import chain, combinations
from itertools import islice
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from joblib import Parallel, delayed
import csv
from pprint import pprint
import math
from operator import itemgetter
import itertools
import scipy
from scipy import stats
from IPython.core.magic import register_line_magic
from IPython.display import HTML, display
import json
import powerlaw
from decimal import Decimal

In [12]:
def get_communities(part,G): # returns the number of communities in G given the partition part
    communities = []
    for i in G.nodes():
        if part[i] not in communities:
            communities.append(part[i])
    return len(communities)

In [19]:
def parity_seeding_IBM(network,S_N,k_p,partition):# return a positive seeed selected accordin gto parity seeding
    S_P_Stoica=set()
    num_com=get_communities(partition,network)
    S_N_population=[0]*num_com
    for i in S_N:
        S_N_population[partition[i]]+=1
        
    degrees_comm={c:[] for c in range(num_com)} #array used to store nodes in descending degree order per community
    for x in list(network.nodes()):
        degrees_comm[partition[x]].append((x,len(list(network.successors(x))))) 
    for c in range(num_com): 
        degrees_comm[c]=[t[0] for t in sorted(degrees_comm[c], key=lambda x: x[1], reverse=True) if  not t[0] in S_N]

    probs=[] # some times the proportional distribution of resources in not exact, we keep track of the remainders to divide any left resources
    taken_s=[0]*num_com # array used to keep track of resources allocated to each community
    for c in range(num_com):
        taken_s[c]=int(S_N_population[c]/len(S_N)*k_p)
        probs.append(S_N_population[c]/len(S_N)*k_p-int(S_N_population[c]/len(S_N)*k_p))

    while np.sum(taken_s)<k_p:
        max_val=max(probs) # if any resources are left we allocate it the the community with the largest remainder
        c_r=probs.index(max_val)
        taken_s[c_r]+=1
        probs[c_r]=0

    for c in range(num_com):
         S_P_Stoica.update(degrees_comm[c][0:taken_s[c]])
    
    return S_P_Stoica

In [20]:
#code to set probability of the edges
def set_weight_degree(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= 1/G.in_degree(v)
    return G

def set_weight_trivalency(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= random.choice([0.1,0.01,0.001])
    return G

def set_weight_value(G,val,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= round(val,3)
    return G

def set_weight_average(G,p_n):
    avg=sum([G.out_degree(x) for x in G.nodes()])/len(G.nodes())
    for u,v in G.edges():
        G[u][v][p_n]= 1/avg
    return G

def set_weight_comm(G,inter,intra,p_n):
    for u,v in G.edges():
        if G.nodes[u]['community']== G.nodes[v]['community']:
            G[u][v][p_n]= inter
        else:
            G[u][v][p_n]= intra
    return G


In [21]:
network=nx.erdos_renyi_graph(1000,0.01,directed=True)


# the network nodes need to have a lavel commuity defining to which community they belong to
partition=community_louvain.best_partition(network.to_undirected())
for node in network.nodes():
    network.nodes[node]['community']=partition[node]




# the network shall also have allocated to each edge the probability of infection        
network=set_weight_degree(network,"positive")
network=set_weight_degree(network,"negative")

k_p=100
k_n=50
S_N=random.choices([v for v in list(network.nodes())],k= k_n)
S_P=parity_seeding_IBM(network,S_N,k_p,partition)
print(S_P)

{3, 8, 9, 10, 528, 19, 24, 28, 30, 547, 37, 557, 55, 56, 70, 585, 587, 77, 596, 598, 91, 610, 101, 102, 108, 112, 631, 119, 123, 639, 648, 137, 140, 672, 161, 171, 685, 689, 177, 696, 192, 709, 203, 214, 215, 217, 218, 733, 223, 738, 227, 749, 240, 766, 259, 260, 270, 273, 274, 276, 794, 283, 295, 830, 837, 327, 844, 851, 855, 347, 861, 352, 355, 872, 363, 365, 367, 371, 886, 378, 395, 908, 401, 404, 924, 933, 423, 425, 434, 451, 974, 978, 982, 471, 986, 479, 480, 494, 500, 506}
